In [1]:
import os
import networkx as nx
import pandas as pd
import numpy as np
import time
import subprocess
import shlex
from Bio.Blast import NCBIXML
from collections import defaultdict
import warnings
from itertools import compress, count, islice
from functools import partial
from operator import eq
import argparse
import logging
import copy
import MHG_partition
warnings.filterwarnings('ignore')

In [2]:
def parseBlastXML(filePath):
    """
    Convert a blast xml file to a dataframe storing all pairwise mapping information
    """ 
    queryList, subjectList, percentIdentityList, alignmentLenList, mismatchList, gapList, qStartList, qEndList, sStartList, sEndList, evalList, bitScoreList, qSeqList, sSeqList = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
    f = open(filePath)
    blast_records = NCBIXML.parse(f)
    for blast_record in blast_records:
        queryAccVer = blast_record.query[:blast_record.query.find(" ")] if blast_record.query.find(" ")!= -1 else blast_record.query
        for alignment in blast_record.alignments:
            subjectAccVer = alignment.hit_def[:alignment.hit_def.find(' ')] if alignment.hit_def.find(" ")!= -1 else alignment.hit_def
            for hsp in alignment.hsps:
                percentIdentity = round(hsp.identities/hsp.align_length*100,3)
                alignLength = hsp.align_length
                mismatch = hsp.align_length - hsp.identities - hsp.gaps
                gaps = hsp.gaps
                qStart = hsp.query_start
                qEnd = hsp.query_end
                sStart = hsp.sbjct_start
                sEnd = hsp.sbjct_end
                evalue = hsp.expect
                bitScore = int(hsp.bits)
                qSeq = hsp.query
                sSeq = hsp.sbjct
                
                queryList.append(queryAccVer)
                subjectList.append(subjectAccVer)
                percentIdentityList.append(percentIdentity)
                alignmentLenList.append(alignLength)
                mismatchList.append(mismatch)
                gapList.append(gaps)
                qStartList.append(qStart)
                qEndList.append(qEnd)
                sStartList.append(sStart)
                sEndList.append(sEnd)
                evalList.append(evalue)
                bitScoreList.append(bitScore)
                qSeqList.append(qSeq)
                sSeqList.append(sSeq)
    df = pd.DataFrame({'queryAccVer':queryList,'subjectAccVer':subjectList, 'identity':percentIdentityList, 'alignmentLength':alignmentLenList,
                      'mismatches':mismatchList, 'gaps':gapList, 'qStart':qStartList,'qEnd':qEndList,'sStart':sStartList,'sEnd':sEndList,
                      'evalue':evalList, 'bitScore':bitScoreList,'qSeq':qSeqList, 'sSeq':sSeqList})
    return df, list(zip(qStartList, qEndList, sStartList, sEndList))

def trim_fully_contain(df, check_list):
    drop_index_list = []
    for l in check_list:
        qStart, qEnd, sStart, sEnd = l[0], l[1], l[2], l[3]
        sub_df = df[(df.qStart >= qStart) & (df.qEnd <= qEnd) & (df.sStart >= sStart) &(df.sEnd <= sEnd)]
        if sub_df.shape[0] > 1:
            sub_df = sub_df.sort_values(by=['alignmentLength'], ascending=False)
            overlap_index_list = list(sub_df.index.values)[1:]
            drop_index_list += overlap_index_list
    df = df.drop(df.index[drop_index_list]).reset_index(drop=True)
    return df

两个blast mapping：A，B；如果B完全是A的subsequence，drop B

In [3]:
df, check_list = parseBlastXML('temp_blastn/G000006925,G000299455.xml')
df = trim_fully_contain(df, check_list)

In [4]:
readyToGraphDf,G = MHG_partition.mhg(df)

2022-07-06 14:46:19,066 - root - INFO - start building dataframe containing pairwise blastn calls
2022-07-06 14:46:20,393 - root - INFO - start building an alignment graph
2022-07-06 14:46:20,825 - root - INFO - alignment graph finished building in time: 1.758699655532837
2022-07-06 14:46:20,825 - root - INFO - starting traversing the alignment graph and MHG partition
2022-07-06 14:46:20,879 - root - INFO - Total 125 cc are waiting to be visited
2022-07-06 14:46:21,737 - root - INFO - cc number 1 is on the show being visited
2022-07-06 14:46:21,737 - root - INFO - cc number 1 is a big cc, big but not that big, but still big; could take a few hours


node B ('G000299455', (4431865, 4470166)) (4458949, 4458987)
blockPath (4458949, 4458987)
leftBoundary 0
rightBoundary 0
subArrayBtoA 
subArrayAtoB 
directionAtoB == directionBtoA
pathAtoB (5557, 5595)
arrayAtoB ATTCAGCGCCTGATGCGACGCTGGCGCGTCTTATCAGGC 39
arrayAtoB count - 0

node B ('G000299455', (4431865, 4470166)) (4463350, 4463392)
blockPath (4463350, 4463392)


UnboundLocalError: local variable 'leftBoundary' referenced before assignment

In [23]:
display(df)
display(readyToGraphDf.sort_values(['alignmentLength'],ascending=False))

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,G000006925,G000299455,98.819,59509,676,27,2397883,2457367,1682614,1623109,0.00000,110276,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...
1,G000006925,G000299455,98.836,58085,670,6,148899,206981,4293324,4235244,0.00000,107742,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...
2,G000006925,G000299455,98.499,53966,777,33,3449767,3503727,488092,434155,0.00000,98973,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...
3,G000006925,G000299455,98.710,45731,539,51,1122423,1168132,3103177,3057477,0.00000,84399,TTGTCAGCAATCCACTACAGGACATTATATGATCAGTTCATTATGG...,TTGTCACTAATCCACTACAGGACATTTTATGATCAGTTCATTATGG...
4,G000006925,G000299455,97.413,41009,991,70,3559547,3600494,376503,335504,0.00000,72612,CGCAGCAGCGTCGATTTCCCGCAGCCAGACGGCCCGACCATCACGA...,CGCAGCAGCGTCGATTTTCCGCAGCCAGACGGCCCGACCATCACGA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30190,G000006925,G000299455,86.486,37,5,0,4476815,4476851,5248465,5248429,3.01304,42,TGCCTGATGCGCTTCGCTTATCAGGCCTACAGCAGGC,TGCCTGATGCACTTCGCTAATCAGGCTTACCGTAGGC
30191,G000006925,G000299455,92.857,28,2,0,1607351,1607378,5295845,5295818,3.01304,42,ATTTGCAGCCGTCAGGCTGCCGTGGTTC,ATTTGCAGCCTCCAGGCTGCCGTGGTTC
30192,G000006925,G000299455,100.000,22,0,0,3714681,3714702,5299284,5299263,3.01304,42,TATTAAACGAGTTAATGATGTC,TATTAAACGAGTTAATGATGTC
30193,G000006925,G000299455,96.000,25,1,0,4818332,4818356,5326126,5326102,3.01304,42,GCCTTCCATGCCGCTGGCGCGGCAT,GCCTTCCATGCCGCTGTCGCGGCAT


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,...,bitScore,qSeq,sSeq,qPair,sPair,qEdge,sEdge,sourceNode,destNode,align
13085,G000006925,G000299455,98.819,59509,676,27,2397883,2457367,1682614,1623109,...,110276,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...,"(2397883, 2457367)","(1682614, 1623109)","((2397883, 2457367), CCGACTGGAATCACTGGATCGAAGT...","((1682614, 1623109), CCGACTGGAATCACTGGATCGAAGT...","(G000006925, (2374481, 2472008))","(G000299455, (1623109, 1682614))","(((G000006925, (2374481, 2472008)), (2397883, ..."
1057,G000006925,G000299455,98.836,58085,670,6,148899,206981,4293324,4235244,...,107742,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...,"(148899, 206981)","(4293324, 4235244)","((148899, 206981), GGCGATGGGATTGATATTTATCTGCGG...","((4293324, 4235244), GGCGATGGGATTGATATTTATCTGC...","(G000006925, (64491, 216702))","(G000299455, (4235244, 4293324))","(((G000006925, (64491, 216702)), (148899, 2069..."
19862,G000006925,G000299455,98.499,53966,777,33,3449767,3503727,488092,434155,...,98973,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...,"(3449767, 3503727)","(488092, 434155)","((3449767, 3503727), GCCGCGAAGAGAAACAGCACTCCTG...","((488092, 434155), GCCGCGAAGAGAAACAGCACTCCTGCG...","(G000006925, (3423912, 3503727))","(G000299455, (434155, 488092))","(((G000006925, (3423912, 3503727)), (3449767, ..."
7995,G000006925,G000299455,98.710,45731,539,51,1122423,1168132,3103177,3057477,...,84399,TTGTCAGCAATCCACTACAGGACATTATATGATCAGTTCATTATGG...,TTGTCACTAATCCACTACAGGACATTTTATGATCAGTTCATTATGG...,"(1122423, 1168132)","(3103177, 3057477)","((1122423, 1168132), TTGTCAGCAATCCACTACAGGACAT...","((3103177, 3057477), TTGTCACTAATCCACTACAGGACAT...","(G000006925, (1122423, 1168132))","(G000299455, (3057394, 3103258))","(((G000006925, (1122423, 1168132)), (1122423, ..."
20290,G000006925,G000299455,97.413,41009,991,70,3559547,3600494,376503,335504,...,72612,CGCAGCAGCGTCGATTTCCCGCAGCCAGACGGCCCGACCATCACGA...,CGCAGCAGCGTCGATTTTCCGCAGCCAGACGGCCCGACCATCACGA...,"(3559547, 3600494)","(376503, 335504)","((3559547, 3600494), CGCAGCAGCGTCGATTTCCCGCAGC...","((376503, 335504), CGCAGCAGCGTCGATTTTCCGCAGCCA...","(G000006925, (3559544, 3600494))","(G000299455, (335504, 377237))","(((G000006925, (3559544, 3600494)), (3559547, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16403,G000006925,G000299455,100.000,22,0,0,2980037,2980058,443221,443242,...,42,AAAAAGGCGCCGAAGCGCCTTT,AAAAAGGCGCCGAAGCGCCTTT,"(2980037, 2980058)","(443221, 443242)","((2980037, 2980058), AAAAAGGCGCCGAAGCGCCTTT)","((443221, 443242), AAAAAGGCGCCGAAGCGCCTTT)","(G000006925, (2976303, 3015537))","(G000299455, (434155, 488092))","(((G000006925, (2976303, 3015537)), (2980037, ..."
16394,G000006925,G000299455,100.000,22,0,0,2979965,2979986,146185,146164,...,42,CCCCTCACCCTAACCCTCTCCC,CCCCTCACCCTAACCCTCTCCC,"(2979965, 2979986)","(146185, 146164)","((2979965, 2979986), CCCCTCACCCTAACCCTCTCCC)","((146185, 146164), CCCCTCACCCTAACCCTCTCCC)","(G000006925, (2976303, 3015537))","(G000299455, (137973, 154383))","(((G000006925, (2976303, 3015537)), (2979965, ..."
28537,G000006925,G000299455,100.000,22,0,0,4572762,4572783,1538554,1538575,...,42,ATTGTCATTCTGATGGGATTAG,ATTGTCATTCTGATGGGATTAG,"(4572762, 4572783)","(1538554, 1538575)","((4572762, 4572783), ATTGTCATTCTGATGGGATTAG)","((1538554, 1538575), ATTGTCATTCTGATGGGATTAG)","(G000006925, (4557883, 4573403))","(G000299455, (1535847, 1550961))","(((G000006925, (4557883, 4573403)), (4572762, ..."
13782,G000006925,G000299455,100.000,22,0,0,2514252,2514273,3091941,3091920,...,42,GATAAGGCGTTTACGCCGCATC,GATAAGGCGTTTACGCCGCATC,"(2514252, 2514273)","(3091941, 3091920)","((2514252, 2514273), GATAAGGCGTTTACGCCGCATC)","((3091941, 3091920), GATAAGGCGTTTACGCCGCATC)","(G000006925, (2479150, 2517237))","(G000299455, (3057394, 3103258))","

In [21]:
for cc in nx.strongly_connected_components(G):
    S = G.subgraph(cc)
    nodeList = list(S.nodes())
    edgeList = list(S.edges())
    numberOfNodes = len(nodeList)
    numberOfEdges = len(edgeList)
    print("number of Nodes:",numberOfNodes)
    print("number of Edges:",numberOfEdges)
    print("")

number of Nodes: 1479
number of Edges: 59624

number of Nodes: 37
number of Edges: 72

number of Nodes: 2
number of Edges: 2

number of Nodes: 2
number of Edges: 2

number of Nodes: 4
number of Edges: 6

number of Nodes: 3
number of Edges: 4

number of Nodes: 4
number of Edges: 6

number of Nodes: 12
number of Edges: 50

number of Nodes: 4
number of Edges: 6

number of Nodes: 3
number of Edges: 4

number of Nodes: 3
number of Edges: 4

number of Nodes: 6
number of Edges: 10

number of Nodes: 21
number of Edges: 44

number of Nodes: 3
number of Edges: 4

number of Nodes: 5
number of Edges: 10

number of Nodes: 5
number of Edges: 8

number of Nodes: 5
number of Edges: 12

number of Nodes: 5
number of Edges: 12

number of Nodes: 13
number of Edges: 44

number of Nodes: 9
number of Edges: 16

number of Nodes: 2
number of Edges: 2

number of Nodes: 2
number of Edges: 2

number of Nodes: 2
number of Edges: 2

number of Nodes: 6
number of Edges: 16

number of Nodes: 3
number of Edges: 4

numb

In [32]:
for l in check_list:
    qStart, qEnd, sStart, sEnd = l[0], l[1], l[2], l[3]
    sub_df = df[(df.qStart <= qStart) & (df.qEnd >= qStart)]
    if sub_df.shape[0] > 1:
        display(sub_df)

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,G000006925,G000299455,98.819,59509,676,27,2397883,2457367,1682614,1623109,0.0,110276,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...,CCGACTGGAATCACTGGATCGAAGTGAGCTGCGTAAACAGTTTTCC...
517,G000006925,G000299455,95.433,1489,52,16,2396403,2397891,1684258,1682786,0.0,2404,AGGGAAAGCAATTCTGACAGCGCCCCCTATACGTTATTAAAAAACT...,AGGGAAAGCAATTCTGACAGCGCCCCCTATACGTTATTAAAAAACT...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
1,G000006925,G000299455,98.836,58085,670,6,148899,206981,4293324,4235244,0.0,107742,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...,GGCGATGGGATTGATATTTATCTGCGGGAATTTATATTATGTTTCT...
740,G000006925,G000299455,98.972,778,8,0,148126,148903,4589518,4588741,0.0,1450,TGTTGGGTAATGCTGCCAACTTACTGATTTAGTGTATGATGGTGAT...,TGTCGGGTAATGCTGCCAACTTACTGATTTAGTGTATGATGGTGAT...
1618,G000006925,G000299455,90.519,770,73,0,148130,148899,1581876,1582645,0.0,1060,GGGTAATGCTGCCAACTTACTGATTTAGTGTATGATGGTGATTTTA...,GGGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTTG...
1701,G000006925,G000299455,90.402,771,74,0,148129,148899,91526,92296,0.0,1056,TGGGTAATGCTGCCAACTTACTGATTTAGTGTATGATGGTGATTTT...,TGGGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTT...
2001,G000006925,G000299455,89.754,771,67,12,148129,148899,5323358,5324116,0.0,1008,TGGGTAATGCTGCCAACTTACTGATTTAGTGTATGATGGTGATTTT...,TGGGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTT...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
2,G000006925,G000299455,98.499,53966,777,33,3449767,3503727,488092,434155,0.0,98973,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...,GCCGCGAAGAGAAACAGCACTCCTGCGAGTAAAAAATCGGAACCTT...
606,G000006925,G000299455,97.721,1009,23,0,3448765,3449773,489123,488115,0.0,1808,TCATCAGCAACGGCGCCATCTGGTCGCCCTGGAATAAACGTTGTGA...,TCACCAGCAACAGCGCCATCTGGTCACCCTGGAATAAACGTTGTGA...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
4,G000006925,G000299455,97.413,41009,991,70,3559547,3600494,376503,335504,0.000000e+00,72612,CGCAGCAGCGTCGATTTCCCGCAGCCAGACGGCCCGACCATCACGA...,CGCAGCAGCGTCGATTTTCCGCAGCCAGACGGCCCGACCATCACGA...
9098,G000006925,G000299455,95.000,40,2,0,3559544,3559583,5248648,5248609,3.416360e-07,66,GTACGCAGCAGCGTCGATTTCCCGCAGCCAGACGGCCCGA,GTACGCAGCAGCGTCGATTTACCGCAGCCGGACGGCCCGA
27472,G000006925,G000299455,84.091,44,7,0,3559547,3559590,3540284,3540241,7.947270e-01,44,CGCAGCAGCGTCGATTTCCCGCAGCCAGACGGCCCGACCATCAC,CGCAGCAGGGTCGATTTACCGGAACCGGACGGCCCGATAATCAC


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
6,G000006925,G000299455,99.065,37656,329,23,3362736,3400369,578411,540757,0.0,70332,CAACAATGTTCGGCATAATTATATCTGTCATCGTATTAATTACGAT...,CAACAATGTTCGGCATAATTATATCTGTCATCGTATTAATTACGAT...
1203,G000006925,G000299455,90.909,770,70,0,3361967,3362736,942497,941728,0.0,1077,AGGTAATGCTGCCAACTTATTGATTTAGTGTATGATGGTGATTTTA...,AGGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTTG...
1422,G000006925,G000299455,90.661,771,72,0,3361966,3362736,3679297,3680067,0.0,1067,AAGGTAATGCTGCCAACTTATTGATTTAGTGTATGATGGTGATTTT...,AAGGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTT...
1472,G000006925,G000299455,90.637,769,72,0,3361968,3362736,696787,696019,0.0,1063,GGTAATGCTGCCAACTTATTGATTTAGTGTATGATGGTGATTTTAA...,GGTGATGCTGCCAACTTACTGATTTAGTGTATGATGGTGTTTTTGA...


...

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
90,G000006925,G000299455,99.089,13057,112,7,2602941,2615991,1444017,1430962,0.0,24390,TCGGGCTATTTATTTACGTCAGGAATCCTGAAAAATCAGCCCGGCG...,TCGGGCTATTTATTTACGTCAGGAATCCTGAAAAATCAGCCCGGCG...
904,G000006925,G000299455,98.060,773,14,1,2602175,2602947,1444726,1443955,0.0,1390,ATGGCGATGTCCGCTGGCGTGTTTATGGCGATCGGATTTACTTTTT...,ATGGCGATGTCCGCTGGCGTGTTTATGGCGATCGGATTTACTTTTT...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
91,G000006925,G000299455,98.703,13187,170,1,2313327,2326513,1765832,1752647,0.000000e+00,24359,TAGCCCGATGGCATCACCATCGGGCCTCTTTTTATTTACTCTCCTG...,TAGCCCGATGGCATCACCATCGGGCCTCTTTTTATTTACTCTCCTG...
3248,G000006925,G000299455,87.500,152,17,2,2313177,2313327,3816019,3815869,1.036190e-36,164,GAGATTTATGACAAACGCAAAACTGCCTGATGCGCTACGCTTATCA...,GAGACTGATGACAAACGCTAAACTGCCTGATGCGCTACGCTTATCA...
3468,G000006925,G000299455,85.714,140,19,1,2313188,2313327,988656,988794,6.357800e-31,144,CAAACGCAAAACTGCCTGATGCGCTACGCTTATCAGGCCTACGTGG...,CAAAAGCAAAATTGCCTGATGCGCTACGCTTATCAGGCCTACACGT...
3784,G000006925,G000299455,81.944,144,26,0,2313184,2313327,5069776,5069919,1.028930e-25,127,ATGACAAACGCAAAACTGCCTGATGCGCTACGCTTATCAGGCCTAC...,ATGACAAGCGTATGATTGCCTGATGCGCTGCGCTTATCAGCCCTAC...
3803,G000006925,G000299455,81.944,144,26,0,2313185,2313328,4656075,4655932,1.028930e-25,127,TGACAAACGCAAAACTGCCTGATGCGCTACGCTTATCAGGCCTACG...,TGATAAATGCAATACTGCCTGATGCGCTATGCCGATCAGGCCTACG...
3858,G000006925,G000299455,85.075,134,18,2,2313200,2313332,4689306,4689438,1.478980e-24,123,TGCCTGATGCGCTACGCTTATCAGGCCTACGTGGATCGAT-CAATT...,TGCCTGATGCACTACGTTTATCCGGCCTACATG-ATCTCTGCAATA...
5073,G000006925,G000299455,86.207,87,11,1,2313241,2313327,1147473,1147558,3.873670e-14,89,AATTTATTGAATTTACACAATTTTGTAGGCCGGATAAGGCGTTCAC...,AATTCATTGAATTTAATTGGTTTTGTAGGCCGGATAAGGCGTTAAC...
5395,G000006925,G000299455,79.114,158,30,3,2313176,2313331,3660729,3660885,5.567960e-13,85,TGAGATTTATGACAAACGCAAAACTGCCTGATGCGCTACGCTTATC...,TGAGAGTGAAGATAAACGCAAAATTACCTGATGCGCTACGCTTGCC...
12642,G000006925,G000299455,82.540,63,11,0,2313265,2313327,953026,953088,7.058480e-05,58,GTAGGCCGGATAAGGCGTTCACGCCGCATCCGGCATAAACTAAGCG...,GTAGGCGGGATAAGGTGTTTACGCCGCATCCGGCATGGAAAACGCG...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
83,G000006925,G000299455,98.743,13370,164,4,3040656,3054024,956197,942831,0.0,24709,TTACTTAATCACCCCATTGCGCCGCAGTTGTTCGGTATCTTTTTTC...,TTACTTAATCACCCCATTGCGCCGCAGTTGTTCGGTATCTTTTTTC...
92,G000006925,G000299455,99.897,12670,12,1,3052415,3065083,4735655,4748324,0.0,24276,GAAATTTGATTGACGAGACGAGGCGAATCAGGTTTAATGCGCCCCG...,GAAATTAGGTTGACGAGATGTGCAGATTACGGTTTAATGCGCCCCG...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
93,G000006925,G000299455,98.078,13269,241,14,4103083,4116338,5060698,5047431,0.0,24022,GGATATTGCGGCCACTCATCGGTCAGTTGATTACGCAGGGCGTAAA...,GGATATTGCGGCCACTCATCGGTCAGTTGATTACGCAGGGCGTAAA...
737,G000006925,G000299455,99.098,776,7,0,4102314,4103089,4588745,4589520,0.0,1452,TGGTAATGACTCCAACTTACTGATAGTGTTTTATGTTCAGATAATG...,TGGTAATGACTCCAACTTACTGATAGTGTTTTATGTTCAGATAATG...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
98,G000006925,G000299455,99.138,12411,87,20,1815406,1827803,2737538,2749941,0.0,23197,GAACGGTTTAAACCTGATTTTGCCCGGACATTGCGCCCCGGCTGGC...,GAACGGTTTAAACCTGATTTTGCCCGGACATTGCGCCCCGGCTGGC...
510,G000006925,G000299455,98.492,1393,21,0,1815406,1816798,2270570,2269178,0.0,2557,GAACGGTTTAAACCTGATTTTGCCCGGACATTGCGCCCCGGCTGGC...,GAACGGTTTAAACCTGATTTTGCCCGGACATTGCGCCCCGGCTGGC...


In [25]:
for l in check_list:
    qStart, qEnd, sStart, sEnd = l[0], l[1], l[2], l[3]
    sub_df = df[(df.sStart <= sStart) & (df.sEnd >= sEnd)]
    if sub_df.shape[0] > 1:
        display(sub_df)

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,G00009306,G90004803,84.447,2919,411,43,334121,337010,663556,660652,0.0,2715,TCTCTCGGATCATTAGTACTGGTTAGCTCAACGCATCTCTACGCTT...,TCTCACGGGTCATTAGTATCGGTTAGCTCAACGCTTCACAACGCTT...
1,G00009306,G90004803,84.447,2919,411,43,35427,38316,663556,660652,0.0,2715,TCTCTCGGATCATTAGTACTGGTTAGCTCAACGCATCTCTACGCTT...,TCTCACGGGTCATTAGTATCGGTTAGCTCAACGCTTCACAACGCTT...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
0,G00009306,G90004803,84.447,2919,411,43,334121,337010,663556,660652,0.0,2715,TCTCTCGGATCATTAGTACTGGTTAGCTCAACGCATCTCTACGCTT...,TCTCACGGGTCATTAGTATCGGTTAGCTCAACGCTTCACAACGCTT...
1,G00009306,G90004803,84.447,2919,411,43,35427,38316,663556,660652,0.0,2715,TCTCTCGGATCATTAGTACTGGTTAGCTCAACGCATCTCTACGCTT...,TCTCACGGGTCATTAGTATCGGTTAGCTCAACGCTTCACAACGCTT...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
2,G00009306,G90004803,87.744,1330,156,7,337282,338609,659934,658610,0.0,1559,TTGTTTTTTTAAGGAGGTGATCCAACCACAGGTTCCCCTACGGTTA...,TTGTTTTAGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTA...
3,G00009306,G90004803,87.744,1330,156,7,38665,39992,659934,658610,0.0,1559,TTGTTTTTTTAAGGAGGTGATCCAACCACAGGTTCCCCTACGGTTA...,TTGTTTTAGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTA...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
2,G00009306,G90004803,87.744,1330,156,7,337282,338609,659934,658610,0.0,1559,TTGTTTTTTTAAGGAGGTGATCCAACCACAGGTTCCCCTACGGTTA...,TTGTTTTAGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTA...
3,G00009306,G90004803,87.744,1330,156,7,38665,39992,659934,658610,0.0,1559,TTGTTTTTTTAAGGAGGTGATCCAACCACAGGTTCCCCTACGGTTA...,TTGTTTTAGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTA...


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
11,G00009306,G90004803,73.123,1012,272,0,13726,14737,507506,506495,1.224880e-102,377,ATGTTTAAAAAAATTAGAGGAATCTTTTCGAATGATTTATCCATCG...,ATGTTCAAAAAATTTCGTGGCATATTTTCTAATGACTTATCTATCG...
12,G00009306,G90004803,73.035,1005,271,0,2906,3910,506495,507499,2.530690e-100,369,TATCTATCATCTCCAAAGCTTTCCCACCGCCCCTTGCGACACAAGT...,TATCGATCATTTCTAATGCCTTTCCTCCACCACGAGCAACACAAGT...


...

,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
438,G00009306,G90004803,92.593,27,2,0,240349,240375,796074,796100,0.237561,41,CTAAGTCATTATTTTTTTTGAAAAATT,CTAAGTAATTATTTTTTTGGAAAAATT
820,G00009306,G90004803,92.000,25,2,0,352711,352735,796099,796075,3.414670,37,ATTTTTCCAAAAAAATAAGAACTTA,ATTTTTCCAAAAAAATAATTACTTA


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
834,G00009306,G90004803,100.0,19,0,0,12822,12840,832422,832440,3.41467,37,ATGTTCGTAATGTGAATTT,ATGTTCGTAATGTGAATTT
835,G00009306,G90004803,100.0,19,0,0,4802,4820,832440,832422,3.41467,37,AAATTCACATTACGAACAT,AAATTCACATTACGAACAT


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
857,G00009306,G90004803,87.097,31,4,0,572127,572157,903984,903954,3.41467,37,AAATTTTAATATTTCTAAATATGAAAATATA,AAATTTTAAAATATCTATATATGCAAATATA
858,G00009306,G90004803,87.097,31,4,0,557542,557572,903954,903984,3.41467,37,TATATTTTCATATTTAGAAATATTAAAATTT,TATATTTGCATATATAGATATTTTAAAATTT


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
221,G00009306,G90004803,80.488,82,13,3,224305,224386,908224,908302,0.000006,56,CGGGAAGCGAGACTTGAACTCGCAAAACCATTTTTGGTGCTAGATC...,CGGGAGGCGAGACTTGAACTCGCAAACC---TTACGGCGCCAGAAC...
859,G00009306,G90004803,95.455,22,1,0,503346,503367,908277,908298,3.414670,37,TGCGTTTACCAATTTCGCCACT,TGCGTCTACCAATTTCGCCACT


,queryAccVer,subjectAccVer,identity,alignmentLength,mismatches,gaps,qStart,qEnd,sStart,sEnd,evalue,bitScore,qSeq,sSeq
860,G00009306,G90004803,89.286,28,3,0,5446,5473,909407,909434,3.41467,37,AAAATTAGATATAGAGAATGTTTAAAAA,AAACTTAGATATAGAAAATTTTTAAAAA
861,G00009306,G90004803,89.286,28,3,0,12169,12196,909434,909407,3.41467,37,TTTTTAAACATTCTCTATATCTAATTTT,TTTTTAAAAATTTTCTATATCTAAGTTT


看一眼 blastn有没有overlapping region in seq A mapped to two subseqs in seq B